## Setup steps
Here are the steps to setup this lab:
- Install missing dependencies and restart the notebook
- Create the notebook variables
- Create the `db_ybu` database

### Install missing dependencies and restart the notebook
Run the following cell to ensure that the notebook dependencies are available to the notebook. 

In [ ]:
!pip install ipython-sql
!pip install psycopg2-binary
!pip install sqlalchemy 

> Important!
> 
> Restart the Notebook.
> 
> Do NOT skip this step.
> 
> After restarting the notebook, you can continue running notebook cells below, at **Create the notebook variables**.


### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip running this cell. 
> 

The following Python cell creates and stores variables that all the notebooks in this lab will use. You can view these variables in the Jupyter tab.

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.
- Verify the accuracy of the output values

In [ ]:
# Env variables for Notebook
import os

# read env_vars.env
env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 

# Comment out Local
MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
MY_GITPOD_WORKSPACE_URL=os.environ.get('MY_GITPOD_WORKSPACE_URL_LOCAL')
MY_SUDO=os.environ.get('MY_SUDO')

# Gitpod specific
# MY_YB_PATH=os.environ.get('MY_YB_PATH')
# MY_GITPOD_WORKSPACE_URL=os.environ.get('GITPOD_WORKSPACE_URL')

# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')
MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')
MY_DATA3_DDL_FILE=os.environ.get('MY_DATA3_DDL_FILE')
MY_DATA3_DML_FILE=os.environ.get('MY_DATA3_DML_FILE')
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
MY_UTIL_YBTSERVER_METRICS_FILE=os.environ.get("MY_UTIL_YBTSERVER_METRICS_FILE")

# Current directory of project and related child folders
MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'
MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'

print(MY_NOTEBOOK_DATA_FOLDER, MY_NOTEBOOK_UTILS_FOLDER)
# Store the note book values for other notebooks to use

%store MY_DB_NAME
%store MY_YB_PATH
%store MY_GITPOD_WORKSPACE_URL
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_NOTEBOOK_DIR
%store MY_TSERVER_WEBSERVER_PORT
%store MY_NOTEBOOK_DATA_FOLDER
%store MY_NOTEBOOK_UTILS_FOLDER
%store MY_DATA3_DDL_FILE
%store MY_DATA3_DML_FILE
%store MY_UTIL_FUNCTIONS_FILE
%store MY_UTIL_YBTSERVER_METRICS_FILE
%store MY_SUDO

In [ ]:
%%bash -s "$MY_SUDO"  # ifconfig aliases
MY_SUDO=${1}

if ifconfig lo0 | grep 127.0.0.[2-7] > /dev/null
then
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.2
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.3
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.4
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.5
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.6
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.7
fi

echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.2
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.3
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.4
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.5
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.6
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.7

echo ${MY_SUDO} | sudo ifconfig lo0

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH

### Grep port 9000 for conflict
# lsof -nP -iTCP -sTCP:LISTEN | grep 9000

# Stop running cluster
if  pgrep -x "yb-tserver" > /dev/null 
then
    ./bin/yb-ctl stop
    sleep 1
fi

# Destroy cluster
if echo `./bin/yb-ctl status` | grep "Node Count"  > /dev/null 
then
    ./bin/yb-ctl destroy
    sleep 1
fi

# Create cluster
./bin/yb-ctl --rf 3 create  \
--tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port="${TSERVER_WEBSERVER_PORT}  \
--master_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1" \
--num_shards_per_tserver=1  \
--placement_info "cloud1.region1.zone1,cloud2.region2.zone2,cloud3.region3.zone3" 

# Output status
./bin/yb-ctl status

### Create the `db_ybu` database with `ysqlsh`
Run the following cell to connect to the local host using `ysqlsh`, create the `db_ybu` database, and then list the databases.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# drop and create
./bin/ysqlsh -d yugabyte -c "drop database if exists "${DB_NAME}";"  
./bin/ysqlsh -d yugabyte -c "create database "${DB_NAME}";" 

# list dbs
./bin/ysqlsh -d yugabyte -c "\l"

## Connect to YugabyteDB using the PostgreSQL Driver for Python
The following cells requires:
- Python 3.8+ and psycopg2

### Create tables and loaded data using DDL and DML scripts
In this section of the notebook, you will:
- Create tables with a DDL script
- Load data with a DML script
- Verify the creation of tables and data
- View the DDL for tbl_countries

##### Create tables, load data, and review relations
Run the following cell to execute the DDL and DML scripts using `ysqlsh`.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_DATA_FOLDER" "$MY_DATA3_DDL_FILE" "$MY_DATA3_DML_FILE"   # World Cities
YB_PATH=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

#ls $DATA_FOLDER

COMPANY_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
COMPANY_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}
echo $COMPANY_DDL_PATH
echo $COMPANY_DML_PATH

cd $YB_PATH

# DDL file
./bin/ysqlsh -d ${DB_NAME} -f ${COMPANY_DDL_PATH} >&/dev/null
sleep 1;

# DML file
./bin/ysqlsh -d ${DB_NAME} -f ${COMPANY_DML_PATH} >&/dev/null
sleep 1;

# Describe relations
./bin/ysqlsh -d ${DB_NAME} -c "\d"

##### View DDL for Table partitions
Run the following cell using `ysqlsh` to view a table definition.

> Note
> 
> SQL magic does not support PostgreSQL `psql` commands. In order to execute `psql` commands, the notebook uses bash and `ysqlsh`.



In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\dt"
./bin/ysqlsh -d ${DB_NAME} -c "\d order_changes"
./bin/ysqlsh -d ${DB_NAME} -c "\d order_changes_2019_02"
./bin/ysqlsh -d ${DB_NAME} -c "\d order_changes_2019_03"
./bin/ysqlsh -d ${DB_NAME} -c "\d order_changes_default"

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

# engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

#### Show table row counts
Run the cell below to view the row counts for the tables.

A SQL update can compute the new value and return it without the need to query again. The following adds 100 to the salaries of all employees who are not managers and show the new value

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

-- ./bin/ysqlsh -d ${DB_NAME} -c "\dt"
-- ./bin/ysqlsh -d ${DB_NAME} -c "SELECT * FROM order_changes"
-- ./bin/ysqlsh -d ${DB_NAME} -c "SELECT * FROM order_changes_2019_02"
./bin/ysqlsh -d ${DB_NAME} -c "SELECT * FROM order_changes_2019_03"
./bin/ysqlsh -d ${DB_NAME} -c "SELECT * FROM order_changes_default"

In [ ]:
%%sql

SELECT * FROM order_changes

-- SELECT * FROM order_changes_2019_03 

--  SELECT * FROM order_changes_2019_02 

--  SELECT * FROM order_changes_default


## Tablespaces and Geo Row Partitioning

In [ ]:
%%sql

CREATE TABLE transactions (
  user_id       INT NOT NULL,
  account_id	    INT NOT NULL,
  geo_partition TEXT,
  account_type  TEXT NOT NULL,
  amount        NUMERIC NOT NULL,
  created_at    TIMESTAMP DEFAULT NOW()
) PARTITION BY LIST (geo_partition)


## Tablespaces

In [ ]:
%%sql

CREATE TABLESPACE tblspace_us WITH (
  replica_placement='{"num_replicas": 1, "placement_blocks":
  [{"cloud":"cloud1","region":"region1","zone":"zone1","min_num_replicas":1}]}'
)

In [ ]:
%%sql

CREATE TABLESPACE  tblspace_eu WITH (
  replica_placement='{"num_replicas": 1, "placement_blocks":
  [{"cloud":"cloud2","region":"region2","zone":"zone2","min_num_replicas":1}]}'
)

In [ ]:
%%sql

CREATE TABLESPACE tblspace_ap WITH (
  replica_placement='{"num_replicas": 1, "placement_blocks":
  [{"cloud":"cloud3","region":"region3","zone":"zone3","min_num_replicas":1}]}'
)

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  

YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -c "\db+"

Create Table Partitions

In [ ]:
%%sql /* Table Reads */

CREATE TABLE transactions_us PARTITION OF transactions
    (user_id, account_id, geo_partition, account_type, amount, created_at,
    PRIMARY KEY (user_id HASH, account_id, geo_partition))
  FOR VALUES IN ('US') TABLESPACE tblspace_us

In [ ]:
%%sql /* Table Reads */

CREATE TABLE transactions_eu PARTITION OF transactions
    (user_id, account_id, geo_partition, account_type, amount, created_at,
    PRIMARY KEY (user_id HASH, account_id, geo_partition))
  FOR VALUES IN ('EU') TABLESPACE tblspace_eu

In [ ]:
%%sql /* Table Reads */

CREATE TABLE transactions_ap PARTITION OF transactions
    (user_id, account_id, geo_partition, account_type, amount, created_at,
    PRIMARY KEY (user_id HASH, account_id, geo_partition))
  FOR VALUES IN ('India') TABLESPACE tblspace_ap

In [ ]:
%%sql

INSERT INTO transactions  VALUES (1, 100, 'US', 'customer', 100, now())
-- INSERT INTO transactions  VALUES (2, 200, 'EU', 'customer', 200, now())
-- INSERT INTO transactions  VALUES (3, 300, 'India', 'customer', 300, now())



Scenario 4. Indexes
Create and analyze index on the fly

Description
Create a new table and a specific index to avoid table scan and sorts

SQL Statement
Step 1: Create a new demo table with randomly generated rows

GENERATE_SERIES function can generate rows. The following uses it to create a table with 42 rows and a random value from 1 to 10

In [ ]:
%%sql 

create table demo as select generate_series(1,42) num, round(10*random()) val;

Step 2: Create the index “demo_val” on demo table

With the goal to query, for a value, with numbers ordered, in the most efficient way, the following creates an index on “val” (hashed for distribution) and “num” in ascending order

In [ ]:
%%sql

create index demo_val on demo(val hash,num asc);

Step 3: Gather optimizer statistics on table demo

The query planner chooses the best access path when provided with statistics about the data stored in the table. The following gathers those statistics.

In [ ]:
%%sql 

analyze demo;

Step 4: Query the Top-3 numbers for a specific value

The following displays the Top-3 numbers for the value 5

In [ ]:
%%sql

select * from demo where val=5 order by num asc fetch first 3 rows only;

Step 5: Verify that index is leading to faster query execution using explain analyze

When defining an index for a specific access pattern, it is good that the developer verifies that the index is used. The following shows that an “Index Only Scan” was used, without the need for an additional “Sort” operation

In [ ]:
%%sql

explain analyze select * from demo where val=5 order by num fetch first 3 rows only;

Step 6: Clean up the table for this exercise.

To leave the database in the same state as before this exercise, the following removes the demo table created before

In [ ]:
%%sql

drop table if exists demo;

2. Built-in Functions
Learn powerful functions for performing complex database operations with ease

Scenario 1. Window Functions
Compare employees hiring time interval by department using LAG function

LAG is a window function that provides access to the row before the current one. The following SQL statement uses WINDOW to define groups of employees by department, in order of their hiring date. LAG is used to access the previous row in this group, to compare the hiring date interval between two employees. FORMAT builds a text from column values, and COALESCE handles the first hire for which there is no previous row in the group. Without those window functions, this query would have required reading the same table two times.


In [ ]:
%%sql

select
dname,ename,job,
coalesce (
  'hired '||to_char(hiredate -
    lag(hiredate) over (per_dept_hiredate),'999')||' days after '||
    lag(ename) over (per_dept_hiredate),
    format('(1st hire in %L)',dname)
) as "last hire in dept"
from emp join dept using(deptno)
window per_dept_hiredate
  as (partition by dname order by hiredate)
order by dname,hiredate;

Scenario 2. Regexp Matching
List all employees with @gmail or .org in their email addresses
Description
REGEXP performs a pattern match of a string expression. The following lists employees with an e-mail ending in ‘.org’ or a domain starting with ‘gmail.’

In [ ]:
%%sql

select * from emp
where email ~ any ( ARRAY[ '@.*\.org$' , '@gmail\.' ] );

Scenario 3. Arithmetic Date Intervals

The interval data type allows you to store and manipulate a period of time in years, months, days... The following example compares overlapping evaluation periods. A WITH clause defines the evaluation period length depending on the job.

Find employees with overlapping evaluation periods

In [ ]:
%%sql

with emp_evaluation_period as (
 select ename,deptno,hiredate,
 hiredate + case when job in ('MANAGER','PRESIDENT')
 then interval '3 month' else interval '4 weeks'
 end evaluation_end from emp)
select * from emp_evaluation_period e1
 join emp_evaluation_period e2
 on (e1.ename>e2.ename) and (e1.deptno=e2.deptno)
where (e1.hiredate,e1.evaluation_end)
 overlaps (e2.hiredate,e2.evaluation_end);

Scenario 4. CROSSTABVIEW
Description
CROSSTABVIEW is a client command to display rows as columns. The following sums the salaries across jobs and departments and displays them as a cross-table

Display total salary per job and department as a cross-table

In [ ]:
%%sql

select job, dname, sum(sal)
from emp join dept using(deptno)
group by dname, job

Scenario 5. NTILE Function

Split e-mails in 3 groups and format them

In order to send e-mails to all employees in different batches, you will split them into 3 groups using the NTILE function, and format them with the FORMAT function and aggregate them in a comma-separated list with the STRING_AGG function

In [ ]:
%%sql

with groups as (
 select ntile(3) over (order by empno) group_num
 ,* 
 from emp
)
select string_agg(format('<%s> %s',ename,email),', ') 
from groups group by group_num;

Advanced Features
Expand your YSQL skills by completing the following 5 scenarios

Scenario 1. GIN Index on Document
List employees that know SQL
Description
The skills are stored in the semi-structured JSON document. We can query them with @>, ?, ?& and ?| operators. And, for best performance, index them.

SQL Statement
Step 1: Create GIN index on JSON document

GIN indexes can provide fast access to elements inside a JSON document. The following creates an index on the ‘skills’ attributes within ‘other_info’ JSON column

In [ ]:
%%sql 

create index emp_skills on emp using gin((other_info->'skills'));

Step 2: Query the JSON attribute list

SQL queries can navigate into the JSON document with -> and check if an array contains a value with @>. The following searches the employees with the “SQL” skill

In [ ]:
%%sql 

select * from emp where other_info->'skills' @> '"SQL"' ;

Step 3: Use Explain pla to verify that index is used

Thanks to the GIN index, this search doesn’t need to read all documents. The following shows the execution plan being an indexed access path.

In [ ]:
%%sql

explain select * from emp where other_info->'skills' @> '"SQL"' ;

Scenario 2. Text Search
Build a search index on department descriptions

Description
SQL queries can search in the text for some words using the to_tsvector() function to extract a list of words that can be compared. We will find all department descriptions with the words 'responsible' and 'services' in it

SQL Statement
Step 1: Create text search index on the description column

GIN indexes can provide fast access to words inside a text. The following creates an index for the simple-grammar vector of words extracted from the department description

In [ ]:
%%sql

create index dept_description_text_search on dept using gin (( to_tsvector('simple',description) ));

Step 2: Query on description for matching words

The following compares the simple-grammar vector of words extracted from the department description with a word search pattern to find the departments which contain ‘responsible’ and ‘services’ in their description.

In [ ]:
%%sql

select * from dept where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

Step 3: Explain plan to verify that index is used

Thanks to the GIN index, this search doesn’t need to read all rows and text. The following shows the execution plan being an indexed access path

In [ ]:
%%sql

explain select * from dept where to_tsvector('simple',description) @@ to_tsquery('simple','responsible & services');

Scenario 3. Stored Procedures
Transfer commission from one employee to another


Description
A stored procedure can encapsulate a procedural logic into an atomic operation. We will create one in PL/pgSQL, named "commission_transfer", that transfers commission “amount” from “empno1” to “empno2”

SQL Statement
Step 1: Create the procedure for the commission transfer between employees

The procedure has two SQL operations: decrease from “empno1” and add to “empno2”. Plus error checking to raise a custom exception if “empno1” doesn’t have the amount to be transferred

In [ ]:
%%sql

create or replace procedure commission_transfer(empno1 int, empno2 int, amount int) as $$
begin
update emp set comm=comm-commission_transfer.amount
  where empno=commission_transfer.empno1 and comm>commission_transfer.amount;
if not found then raise exception 'Cannot transfer % from %',amount,empno1; end if;
update emp set comm=comm+commission_transfer.amount
  where emp.empno=commission_transfer.empno2;
if not found then raise exception 'Cannot transfer from %',empno2; end if;
end;
$$ language plpgsql;

Step 2: Call the procedure with employee IDs and the amount to be transferred

Once defined, the stored procedure is called with values for all parameters. This transfers 100 from employee 7521 to 7654

In [ ]:
%%sql

call commission_transfer(7521,7654,100);

Step 3: List all employees who have received commission

The following displays all employees having a commission, to verify that 100 have been transferred

In [ ]:
%%sql

SELECT * from emp where comm is not null;

Step 4: Call the procedure with employee IDs and amount that is not allowed to invoke error handling

The following attempts to transfer 1000000, more than what 7521 has. It raises the “Cannot transfer” error defined in the procedure and automatically reverts all intermediate changes to return to a consistent state

In [ ]:
%%sql

call commission_transfer(7521,7654,1000000);

Scenario 4. Triggers
Record the last update time of each row automatically

Description
We will add a column to hold the last update time, and declare a trigger to update it automatically, on the departments table

SQL Statement
Step 1: Add a column to store the last update time

The structure of a SQL table can evolve. With the goal of recording the last update, the following adds a “last_update” column to the department table.

In [ ]:
%%sql

alter table dept add last_update timestamptz;

Step 2: Add a function “dept_last_update” to set the last update time. The following SQL query uses the built-in function transaction_timestamp(), which returns the current date and time at the start of the current transaction

A stored function declares some procedural logic that returns a value. The following returns the “new” state for a trigger after setting the “last_update” to the current time.

In [ ]:
%%sql

create or replace function dept_last_update() returns trigger as $$
begin
  new.last_update:=transaction_timestamp();
  return new;
end;
$$ language plpgsql;

Step 3: Create a trigger “dept_last_update” to call the function “dept_last_update()” on each table update

The previous function can be called automatically. The following trigger executes it on each row update for the departments table.

In [ ]:
%%sql

create trigger dept_last_update
before update on dept
for each row
execute procedure dept_last_update();

Step 4: Display the current state of the table

In order to verify the automatic logging of the last update time, the following displays the current state of departments before any update

In [ ]:
%%sql

select deptno,dname,loc,last_update from dept;

Scenario 5. Materialized Views
View
Pre-compute analytics for reporting, with a materialized view

Description
In order to get fast on-demand reports, we create a materialized view to store pre-joined and pre-aggregated data. This view will store the total salary per department, the number of employees, and the list of jobs in the department.

SQL Statement
Step 1: Create the materialized view

In [ ]:
%%sql

create materialized view report_sal_per_dept as
select 
deptno,dname,
sum(sal) sal_per_dept,
count(*) num_of_employees,
string_agg(distinct job,', ') distinct_jobs
from dept join emp using(deptno)
group by deptno,dname
order by deptno;

Step 2: Indexes can be created on it. This one allows fast queries on a range of total salary

In [ ]:
%%sql

create index report_sal_per_dept_sal on report_sal_per_dept(sal_per_dept desc);

Step 3: A refresh can be scheduled on a daily basis to re-compute it in the background with a simple command.

In [ ]:
%%sql

select *
from report_sal_per_dept
where sal_per_dept<=10000
order by sal_per_dept;

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `db_ybu` database with `ysqlsh`
  - Created utils
  - Created tables and loaded data using DDL and DML scripts
  - Connected to the database using a PostgreSQL driver for Python

Next, run the following cell to open `02_Demystifying_table_sharding_tablets_and_data_distribution.ipynb`.